In [1]:
# Initialize these:
typeImg = "photo" # photo/ cartoon/ all"
numImg = "50" # 5 or 50
sz = 100

In [2]:
# # Include imports
import glob
from skimage.transform import resize
from skimage.io import imread, imsave, imshow
import os
import pandas as pd
import numpy as np
import skimage.color as skcolor
import matplotlib.pyplot as plt

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import time

In [3]:
colNames = ["col_"+str(i) for i in range(58)] 
allConcepts = ['mango','watermelon','honeydew','cantaloupe','grapefruit','strawberry','raspberry','blueberry','avocado','orange','lime','lemon']

# Read data
# Get the color ratings data -From file data_clean.csv
data = pd.read_csv("data_clean.csv",names = colNames)

# Get the LAB coordinates of colors from file LabOrig.csv
#colorData = pd.read_csv("LabOrig.csv",header=None)
colorData = pd.read_csv("Lab.csv",header=None)
colorDataRGB = pd.read_csv("RGB.csv",header=None)

category = pd.read_csv("Category.csv",header=None)
path = "E:/GitRepo/Color-Concept-Associaitons-using-Google-Images/downloads"
# if not os.path.exists( path + "/trainResized" ):
#    os.makedirs( path + "/trainResized/" )


In [4]:
# Count total number of images
count = 0
for i in allConcepts:
    trainFiles = glob.glob( path + "/" +typeImg +"/" +i+"/*.*" )
    count = count + len(trainFiles)
    print(i,count)
print("Category: ",typeImg)
print("Total images : ",count)

mango 50
watermelon 100
honeydew 150
cantaloupe 200
grapefruit 250
strawberry 300
raspberry 350
blueberry 400
avocado 450
orange 500
lime 550
lemon 600
Category:  photo
Total images :  600


In [5]:
import matlab.engine
category = pd.read_csv("Category.csv",header=None)

def getCategoriesMatrix(imageFileName,sz):
    ## Get total pixel counts of all 11 categories for this imageFile
    eng = matlab.engine.start_matlab()
    imageCat = np.array(eng.getCategories(imageFileName,sz))
    eng.quit()
     # imageCat : sz x sz matrix where each pixel has its category
    return imageCat

def getSegmentedImage(imageFileName,sz): # Returns segmented pixels in LAB and its corresponding color category.
    eng = matlab.engine.start_matlab()
    segImg, segImgCat = eng.activeContourSegmentation(imageFileName,sz, nargout=2)
    eng.quit()
    segImg = np.array(segImg)/255;
    segImg = skcolor.rgb2lab(segImg)
    segImgCat = np.array(segImgCat)
    return segImg, segImgCat


In [6]:
numFt = 186
X = np.zeros((count*58,numFt+1) )  # +1 for label #Number of features [] # Feature Vector -- 58N rows
init = time.time()
rowNum = -1
strtInd = []  # To keep track of the starting indices of fruits in the feature matrix
cvInd = 1 
counter = 0
sz = 100    # Size of image: 100 by 100


##
Rc = [1,10,20,30,40]   # Radii thresholds
Li = [1, 10, 20, 30, 40]
R = [1, 10, 20, 30, 40]     # same as L
hueAngle = [5, 10, 20, 30, 40] # Hue Angle
##
for conceptNum,concept in enumerate(allConcepts):
    t0 = time.time()
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>\n",conceptNum+1,": Concept : ",concept)
    # Get label for corresponding fruit (for given color)
    #label = data.loc[conceptNum]['col_'+str(color)]
    ct = 0    # Keeps track of images inside a category
    strtInd.append(counter)
    trainFiles = glob.glob( path + "/" +typeImg +"/" +concept+"/*.*" )
    print("\tTotal number of images for ",concept," : ",len(trainFiles))
    t = []
    for m, nameFile in enumerate(trainFiles):
        t1 = time.time()
        #print("\nReading file ......")
        ct = ct + 1
        
        image = imread( nameFile )
        imageResized = resize( image, (sz,sz),anti_aliasing=True,mode='reflect')
        img = imageResized
        # Convert the pixels in  image to LAB space
        imgLAB = skcolor.rgb2lab(img)
        segmentedImg,segmentedImgCat = getSegmentedImage(nameFile,sz)
        imageWithCategories = getCategoriesMatrix(nameFile,sz)
        
        i1 = imgLAB[27:72,27:72,:] 
        i2 = imgLAB[19:83,19:83,:]
        i3 = imgLAB[12:90,12:90,:]
        i4 = imgLAB[6:96,6:96,:]
        i5 = np.copy(imgLAB)
        i6 = np.copy(segmentedImg)
        
        ii1 = imageWithCategories[27:72,27:72].flatten()
        ii2 = imageWithCategories[19:83,19:83].flatten()
        ii3 = imageWithCategories[12:90,12:90].flatten()
        ii4 = imageWithCategories[6:96,6:96].flatten()
        ii5 = np.copy(imageWithCategories).flatten()
        ii6 = np.copy(segmentedImgCat).flatten()
        
        categoryImg1 = [sum(1 for p in ii1 if p-1 == k)/ii1.size for k in range(11)]
        categoryImg2 = [sum(1 for p in ii2 if p-1 == k)/ii2.size for k in range(11)]
        categoryImg3 = [sum(1 for p in ii3 if p-1 == k)/ii3.size for k in range(11)]
        categoryImg4 = [sum(1 for p in ii4 if p-1 == k)/ii4.size for k in range(11)]
        categoryImg5 = [sum(1 for p in ii5 if p-1 == k)/ii5.size for k in range(11)]
        categoryImg6 = [sum(1 for p in ii6 if p-1 == k)/ii6.size for k in range(11)]
        
        
        for color in range(58):
            rowNum+=1
            counter += 1
            colCt = 0
            # Thus begin the long long process
            # Get spherical and add till 
            colorLAB = np.array(colorData.loc[color])   #get LAB values for specific color
            
            ##   Columns 1-30
            for i in range(6):
                if i == 0:
                    distPixel = skcolor.deltaE_cie76(i1,colorLAB)
                elif i == 1:
                    distPixel = skcolor.deltaE_cie76(i2,colorLAB)
                elif i == 2:
                    distPixel = skcolor.deltaE_cie76(i3,colorLAB)
                elif i == 3:
                    distPixel = skcolor.deltaE_cie76(i4,colorLAB)
                elif i == 4:
                    distPixel = skcolor.deltaE_cie76(i5,colorLAB)
                else:
                    distPixel = skcolor.deltaE_cie76(i6,colorLAB)
                distPixel = distPixel.flatten()
                #print(distPixel.size)
                for j,threshold in enumerate(Rc):
                    num = sum(1 for t in distPixel  if t <= threshold)/distPixel.size
                    #features.append(num)
                    X[rowNum,colCt] = num
                    
                    colCt+=1
            #print(colCt)
            
            # For features 31-180
            for i in range(6):
                if i == 0:
                    L = i1[:,:,0]
                    a = i1[:,:,1]
                    b = i1[:,:,2]
                elif i == 1:
                    L = i2[:,:,0]
                    a = i2[:,:,1]
                    b = i2[:,:,2]
                elif i == 2:
                    L = i3[:,:,0]
                    a = i3[:,:,1]
                    b = i3[:,:,2]
                elif i == 3:
                    L = i4[:,:,0]
                    a = i4[:,:,1]
                    b = i4[:,:,2]
                elif i == 4:
                    L = i5[:,:,0]
                    a = i5[:,:,1]
                    b = i5[:,:,2]
                else:
                    L = i6[:,:,0]
                    a = i6[:,:,1]
                    b = i6[:,:,2]
                    
                LTemp = L.flatten()
                r = np.sqrt(a**2+b**2).flatten()
                theta = np.degrees(np.arctan2(b,a)).flatten()

                L_diff = np.abs(LTemp - colorLAB[0])
                r_diff = np.abs(r - (np.sqrt(colorLAB[1]**2 + colorLAB[2]**2)))
                theta_diff = np.abs(theta - np.degrees(np.arctan2(colorLAB[2],colorLAB[1])))

                for j,r_th in enumerate(R):
                    l_th = Li[j] # L and R are the same.
                    for theta_th in hueAngle:
                        pixels = 0
                        for p in range(len(L_diff)):
                            if (L_diff[p] <= l_th and r_diff[p] <= r_th and theta_diff[p] <= theta_th) or (L_diff[p] <= l_th and r_diff[p] <= 2.5):
                                pixels += 1
                        X[rowNum,colCt] = pixels/(len(L_diff))
                        colCt+=1
            #print(colCt)
            
            colorCat = category.loc[color,0] 
            X[rowNum,colCt] = categoryImg1[colorCat-1]
            colCt+=1
            X[rowNum,colCt] = categoryImg2[colorCat-1]
            colCt+=1
            X[rowNum,colCt] = categoryImg3[colorCat-1]
            colCt+=1
            X[rowNum,colCt] = categoryImg4[colorCat-1]
            colCt+=1
            X[rowNum,colCt] = categoryImg5[colorCat-1]
            colCt+=1
            X[rowNum,colCt] = categoryImg6[colorCat-1]
            colCt+=1
            
            label = data.loc[conceptNum]['col_'+str(color)]
            X[rowNum,colCt] = label
            #print(colCt)
        h = time.time()-t1
        print("\t\t",m, "image: ",h ," seconds")
        t.append(h)
    #print(t)
    print("\t** Time elapsed since last category: ",time.time()-t0," seconds **")
strtInd.append(len(X))
print(strtInd)
print("** Total Time elapsed: ",time.time()-init," seconds **")

>>>>>>>>>>>>>>>>>>>>>>>>>>>>
 1 : Concept :  mango
	Total number of images for  mango  :  50
		 0 image:  169.17466473579407  seconds
		 1 image:  86.49567031860352  seconds
		 2 image:  78.95986342430115  seconds
		 3 image:  80.38903951644897  seconds
		 4 image:  84.13306736946106  seconds
		 5 image:  80.49372243881226  seconds
		 6 image:  83.8986566066742  seconds
		 7 image:  82.35079455375671  seconds
		 8 image:  92.93153858184814  seconds
		 9 image:  84.29755091667175  seconds
		 10 image:  93.09609985351562  seconds
		 11 image:  82.79357481002808  seconds
		 12 image:  78.81525015830994  seconds
		 13 image:  81.95987629890442  seconds
		 14 image:  82.83147287368774  seconds
		 15 image:  82.76668405532837  seconds
		 16 image:  84.0732352733612  seconds
		 17 image:  84.52995824813843  seconds
		 18 image:  87.93685746192932  seconds
		 19 image:  82.6639564037323  seconds
		 20 image:  91.05751276016235  seconds
		 21 image:  85.63800430297852  seconds
		 22 image:  87.

		 35 image:  121.65968203544617  seconds
		 36 image:  101.69407224655151  seconds
		 37 image:  92.66303181648254  seconds
		 38 image:  85.09944415092468  seconds
		 39 image:  93.3204619884491  seconds
		 40 image:  94.41566276550293  seconds
		 41 image:  97.42448854446411  seconds
		 42 image:  96.47204685211182  seconds
		 43 image:  99.69819784164429  seconds
		 44 image:  85.70835447311401  seconds
		 45 image:  103.88894128799438  seconds
		 46 image:  97.09120035171509  seconds
		 47 image:  110.63533782958984  seconds
		 48 image:  95.80580568313599  seconds
		 49 image:  88.7517740726471  seconds
	** Time elapsed since last category:  4652.660230636597  seconds **
>>>>>>>>>>>>>>>>>>>>>>>>>>>>
 5 : Concept :  grapefruit
	Total number of images for  grapefruit  :  50
		 0 image:  94.65145826339722  seconds
		 1 image:  98.38452744483948  seconds
		 2 image:  99.95862793922424  seconds
		 3 image:  108.99812054634094  seconds
		 4 image:  104.75914931297302  seconds
		 5 imag

		 17 image:  94.61252427101135  seconds
		 18 image:  94.16954159736633  seconds
		 19 image:  125.0723705291748  seconds
		 20 image:  114.60369420051575  seconds
		 21 image:  111.53499341011047  seconds
		 22 image:  101.15661954879761  seconds
		 23 image:  109.59603929519653  seconds
		 24 image:  104.5489890575409  seconds
		 25 image:  107.18554043769836  seconds
		 26 image:  93.13901114463806  seconds
		 27 image:  98.1091079711914  seconds
		 28 image:  87.57263231277466  seconds
		 29 image:  145.44109058380127  seconds
		 30 image:  120.00743794441223  seconds
		 31 image:  122.5063967704773  seconds
		 32 image:  138.18614220619202  seconds
		 33 image:  124.78336906433105  seconds
		 34 image:  115.45561408996582  seconds
		 35 image:  125.41530203819275  seconds
		 36 image:  117.16989278793335  seconds
		 37 image:  128.16952514648438  seconds
		 38 image:  120.28498125076294  seconds
		 39 image:  100.22850966453552  seconds
		 40 image:  124.52843022346497  seconds
	

		 1 image:  95.6000189781189  seconds
		 2 image:  103.94414973258972  seconds
		 3 image:  84.32316160202026  seconds
		 4 image:  81.96360397338867  seconds
		 5 image:  91.10214638710022  seconds
		 6 image:  88.06802582740784  seconds
		 7 image:  95.67502164840698  seconds
		 8 image:  103.50220036506653  seconds
		 9 image:  78.88104271888733  seconds
		 10 image:  88.59609317779541  seconds
		 11 image:  83.143714427948  seconds
		 12 image:  85.58809924125671  seconds
		 13 image:  80.937575340271  seconds
		 14 image:  82.57120418548584  seconds
		 15 image:  82.8733971118927  seconds
		 16 image:  82.71486186981201  seconds
		 17 image:  86.1655924320221  seconds
		 18 image:  92.11468863487244  seconds
		 19 image:  80.18359112739563  seconds
		 20 image:  79.93226075172424  seconds
		 21 image:  72.01148343086243  seconds
		 22 image:  83.9504759311676  seconds
		 23 image:  84.75337600708008  seconds
		 24 image:  88.41457748413086  seconds
		 25 image:  85.94418406486511

In [7]:
strtInd

[0,
 2900,
 5800,
 8700,
 11600,
 14500,
 17400,
 20300,
 23200,
 26100,
 29000,
 31900,
 34800]

In [8]:
## convert your array into a dataframe
xTemp = np.copy(X)
#y = np.copy(X[:,-1])
df = pd.DataFrame(xTemp)
## save to xlsx file

filepath = 'dataPhoto50.xlsx'
df.to_excel(filepath, index=False)
tot = len(xTemp)

In [9]:
# for conceptNum,concept in enumerate(allConcepts):
#     t0 = time.time()
#     print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>\n",conceptNum+1,": Concept : ",concept)
#     # Get label for corresponding fruit (for given color)
#     #label = data.loc[conceptNum]['col_'+str(color)]
#     ct = 0    # Keeps track of images inside a category
#     trainFiles = glob.glob( path + "/" +typeImg +"/" +concept+"/*.*" )
#     print("\tTotal number of images for ",concept," : ",len(trainFiles))
#     t = []
#     for m, nameFile in enumerate(trainFiles):
#         print(nameFile)
#         p = imread(nameFile)
#         imageResized = resize( p, (sz,sz),anti_aliasing=True,mode='reflect')
        